In this notebook we are going to create a simple agentic system that will generate ideas given a topic. The system is made up of two agents, named "Idea maker" and "Idea Hater". These agents will create and critizice ideas, and will interact with each other to refine and improve the idea.

The slides associated to this notebook can be found [here](https://docs.google.com/presentation/d/1iuve05pqrZtt4RxbViAl4l8lvLwh3cUKWzT5wrqk6Bg/edit?usp=sharing).

These agents represent a simplified version of the Idea module in [Denario](https://arxiv.org/abs/2510.26887). See [this link](https://astropilot-ai.github.io/DenarioPaperPage/) for more details.  



---



Lets install the relevant packages to create the agentic system:

In [ ]:
!pip install langgraph -q
!pip install langchain -q
!pip install langchain-google-genai -q

In this tutorial we will be using Gemini models. To use them, we need an API key. Lets set the GOOGLE API Key

Go to [this link](https://ai.google.dev/gemini-api/docs/api-key) to get your key if you don't have one already.

In [ ]:
import getpass
GOOGLE_API_KEY = getpass.getpass('Enter your Gemini API key: ')

Before diving into the system, lets see how to call Large Language Models (LLMs) and manipulate their output. For this, lets use Gemini

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.5, google_api_key=GOOGLE_API_KEY)

Lets define the message we will be passing to the LLM. In LangChain/LangGraph, there are three kind of messages:
- HumanMessage: These are messages the user/human will send to the LLM
- AIMessage: These are messages coming from LLM. Typically, the output to a query.
- SystemMessage: These are messages send to the LLM to define its behaviour. For instance: "You are an astrophysicist". Note that these messages may not be supported by all LLMs.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

message = [HumanMessage(content="Tell me something about NASA")]

Now that we have defined the LLM and the prompt, we can invoke it:

In [ ]:
result = llm.invoke(message)

Differently to the straight output we get when using ChatGPT or Gemini in a browser, with LangChain/LangGraph, we get an output with lots of details

In [ ]:
result

To print just the output content do:

In [ ]:
print(result.content)

There are many different fields that are actually very important when designing agentic systems

In [ ]:
result.usage_metadata['output_tokens']

Now, lets move on and start building our agentic system.

Lets define the graph state. This is a python dictionary that contains information that all agents can access, edit, and add. It is critical to establish the communication between agents, and the context/memory of them.

In [ ]:
from typing_extensions import TypedDict, Any
from typing import Annotated, Literal
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class GraphState(TypedDict):
  topic: str
  idea: str
  previous_ideas: str
  previous_critiques: str
  iteration: int

Now lets create the agents for our system. In this case, we only have 2:
- Idea maker: this agent is in charge of generating an idea given a topic and any available feedback
- Idea maker: this agent is in charge or criticizing an idea

In [ ]:
def idea_maker(state: GraphState):

  PROMPT = [HumanMessage(content=f"""Given the topic below generate an interesting idea for a science project. Take into account any critique provided and previous generated ideas, if any:

  Topic:
  {state['topic']}

  Previous ideas:
  {state['previous_ideas']}

  Previous critiques:
  {state['previous_critiques']}
  """)]

  llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.7, google_api_key=GOOGLE_API_KEY)

  result = llm.invoke(PROMPT)
  idea = result.content  #just get the content, not all the other information
  state['iteration'] += 1 #increase the counter by one

  display(HTML("<p style='color:Green'>########### Idea ###########</p>"))
  print(idea)
  display(HTML("<p style='color:Green'>############################</p>"))

  previous_ideas = f"""
  {state['previous_ideas']}

  Iteration {state['iteration']}:
  {idea}
  """

  return {'idea': idea, 'iteration':state['iteration'], 'previous_ideas':previous_ideas}

Now lets create the idea hater agent

In [ ]:
def idea_hater(state: GraphState):

  PROMPT = [HumanMessage(content=f"""Critique the proposed idea in order to improve it. Take into account all previous ideas, and critiques, if any,:

  Topic:
  {state['topic']}

  Current idea:
  {state['idea']}

  Previous ideas:
  {state['previous_ideas']}

  Critiques:
  {state['previous_critiques']}
  """)]

  llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.8, google_api_key=GOOGLE_API_KEY)

  result = llm.invoke(PROMPT)
  critique = result.content

  previous_critiques = f"""
  {state['previous_critiques']}

  Iteration {state['iteration']}:
  {critique}
  """

  display(HTML("<p style='color:red'>########### Critique ###########</p>"))
  print(critique)
  display(HTML("<p style='color:red'>################################</p>"))

  return {'previous_critiques':previous_critiques}

We are going to create this simple python function to define the which node/agent should go after idea maker. In this case, if less than three iterations, the idea generated by idea maker is send to idea hater. After 3 iterations, the system ends.

In [ ]:
# Idea maker - hater router
def router(state: GraphState) -> Literal['hater', '__end__']:

    if state['iteration']<3:
        return "hater"
    else:
        return "__end__"


We now define the computational graph. This defines the agentic workflow, i.e. how agents interact with each other and how the system progresses.

In [ ]:
from langgraph.graph import START, StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display, HTML

# Define the graph
builder = StateGraph(GraphState)

# Define nodes: these do the work
builder.add_node("maker",             idea_maker)
builder.add_node("hater",             idea_hater)

# Define edges: these determine how the control flow moves
builder.add_edge(START,                 "maker")
builder.add_conditional_edges("maker",  router)
builder.add_edge("hater",               "maker")


memory = MemorySaver()
graph  = builder.compile(checkpointer=memory)

# # generate an scheme with the graph
try:
    import requests
    original_post = requests.post

    def patched_post(*args, **kwargs):
        kwargs.setdefault("timeout", 30)  # Increase timeout to 30 seconds
        return original_post(*args, **kwargs)

    requests.post = patched_post
    graph_image = graph.get_graph(xray=True).draw_mermaid_png()
    display(Image(data=graph_image))
except Exception as e:
    print(f"⚠️ Failed to generate or save graph diagram: {e}")

Now we that we have 1) the agents, 2) the graph state, and 3) the computational graph we can run the system.

In [ ]:
input = {'topic':'scaling relations of galaxy clusters',
         'previous_ideas': '',
         'idea': '',
         'previous_critiques': '',
         'iteration': 0}
result = graph.invoke(input, config={"configurable": {"thread_id": "run-001"}})


**Exercise:** Modify the above system to include a final agent that will summarize the conversation between the maker and hater agents